## Neural Style Transfer with OpenVINO™ Toolkit

In [1]:
#https://medium.com/intel-software-innovators/neural-style-transfer-with-open-vino-toolkit-b56ae7c6a2c
#source /opt/intel/openvino/bin/setupvars.sh
import os
import cv2
import time
import numpy as np
from openvino.inference_engine import IENetwork, IEPlugin
from matplotlib import pyplot as plt
%matplotlib inline
print("Imported Python modules.")

Imported Python modules.


### Configuration Params

In [2]:
# model IR files
model_xml = "./nst_vgg19/nst_vgg19-symbol.xml"
model_bin = "./nst_vgg19/nst_vgg19-symbol.bin"
# input image file
input_path = "../images/tubingen.jpg"
# CPU extension library to use
cpu_extension_path = "libcpu_extension.so"
# device to use
device = "MYRIAD"
# RGB mean values to add to results
mean_val_r = 123.68
mean_val_g = 116.779
mean_val_b = 103.939
print("Configuration parameters settings:"
"\n\tmodel_xml=", model_xml,
"\n\tmodel_bin=", model_bin,
"\n\tinput_path=", input_path,
"\n\tcpu_extension_path=", cpu_extension_path,
"\n\tdevice=", device,
"\n\tmean_val_r=", mean_val_r,
"\n\tmean_val_g=", mean_val_g,
"\n\tmean_val_b=", mean_val_b )

Configuration parameters settings:
	model_xml= ./nst_vgg19/nst_vgg19-symbol.xml 
	model_bin= ./nst_vgg19/nst_vgg19-symbol.bin 
	input_path= ../images/tubingen.jpg 
	cpu_extension_path= libcpu_extension.so 
	device= MYRIAD 
	mean_val_r= 123.68 
	mean_val_g= 116.779 
	mean_val_b= 103.939


### Create a Plugin for Device

In [3]:
# create plugin for device
plugin = IEPlugin(device=device)
print("A plugin object has been created for device [", plugin.device, "]")

A plugin object has been created for device [ MYRIAD ]


### Create Network from Model IR

In [4]:
# load network from IR files
net = IENetwork.from_ir(model=model_xml, weights=model_bin)
print("Loaded model IR files [",model_bin,"] and [", model_xml, "]\n")

Loaded model IR files [ ./nst_vgg19/nst_vgg19-symbol.bin ] and [ ./nst_vgg19/nst_vgg19-symbol.xml ]



/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: from_ir() method of IENetwork is deprecated. Please use IENetwork class constructor to create valid IENetwork instance
  


### Load model into Device Plugin

In [5]:
# load the model into the plugin
exec_net = plugin.load(network=net, num_requests=2)
# store name of input and output blobs
input_blob = next(iter(net.inputs))
output_blob = next(iter(net.outputs))
# read the input's dimensions: n=batch size, c=number of channels, h=height, w=width
n, c, h, w = net.inputs[input_blob].shape
print("Loaded model into plugin.  Model input dimensions: n=",n,", c=",c,", h=",h,", w=",w)

Loaded model into plugin.  Model input dimensions: n= 1 , c= 3 , h= 224 , w= 224


### Prepare input image

In [6]:
# define function for resizing input image
def resizeInputImage(image):
    # resize image dimensions form image to model's input w x h
    in_frame = cv2.resize(image, (w, h))
    # Change data layout from HWC to CHW
    in_frame = in_frame.transpose((2, 0, 1))
    # reshape to input dimensions
    in_frame = in_frame.reshape((n, c, h, w))
    return in_frame

def processResults(res):
    # get output
    result = res[output_blob][0]
    # Change layout from CxHxW to HxWxC
    result = np.swapaxes(result, 0, 2)
    result = np.swapaxes(result, 0, 1)
    # add RGB mean values to
    result = result[::] + (mean_val_r, mean_val_g, mean_val_b)
    # Clip RGB values to [0, 255] range
    result[result < 0] = 0
    result[result > 255] = 255
    # Matplotlub expects normilized image with pixel RGB values in range [0,1].
    result = result / 255
    return result

# create function to process and display inference results
def processAndDisplayResults(res, orig_input_image, orig_input_path, verbose = True):
    # display original input image
    plt.figure()
    plt.axis("off")
    im_to_show = cv2.cvtColor(orig_input_image, cv2.COLOR_BGR2RGB)
    plt.imshow(im_to_show)

### Run Inference

In [7]:
import cv2
cap = cv2.VideoCapture(<video>)
while(cap.isOpened()):
    # grab the frame from the threaded video stream
    ret, frame = cap.read()
    o_image = cv2.resize(frame, (w, h))
    # resize frame
    in_frame = resizeInputImage(frame)
    # inference
    res = exec_net.infer(inputs={input_blob: in_frame})
    # convert back to image
    image = processResults(res)
    cv2.imshow("Input", o_image)
    cv2.imshow('Output',image)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
        
# do a bit of cleanup
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.1.2-openvino) /localdisk/jenkins/workspace/OpenCV/OpenVINO/build/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
